In [66]:
import pandas as pd
import openpyxl as xl
from shutil import copy

Staff = pd.read_excel('CopyШтат.xlsx')  # считываем файл со штатом сотрудников = Staff

Staff['full_name'] = (Staff["Фамилия"] + " " + Staff["Имя"] + " " + Staff["Отчество"]).astype("string") #делаем full_name - ФИО

departments = Staff['Подразделение'].unique() #находим уникальные значения, то есть все департменты 

In [67]:
# можно здесь что-то подправить, если шаблон измениться 
template_size = 50 # ppl per file (кол-во людей, которое может быть в одном файле, если окажется что в департменте работает больше, просто изменить значение )
first_row = 14     # first row of the empty list in the template

#здесь как раз и отлавливаю ошибку, если вдруг окажется что лист очень маленький, а народу в департменте много
max_people = Staff.groupby('Подразделение')['full_name'].count().max()
if template_size < max_people:
    raise ValueError('The template is too small!')

last_row = first_row + template_size * 2 - 1  # последняя пустая строчка в  template

print(f'Check that the empty list in template.xlsx goes from row {first_row} to row {last_row}') #для дебага, и понимания того что происходит 

Check that the empty list in template.xlsx goes from row 14 to row 113


In [69]:
for department in departments:

    # создаем файл для каждого департмента 
    file = f'{department}.xlsx'
    copy('template.xlsx', file) #название можно поменять
    wb = xl.load_workbook(file)
    sheet = wb['Табель'] #название листа, можно поменять на нужный в шаблоне на входе (то есть в template.xlsx )
    sheet['B6'] = department #добавляем название департмента в ячейку B6

    # добавляем всех сотрудников в файл
    people = Staff.loc[Staff["Подразделение"] == department,                # фильтр по отделу
                       ['full_name', 'Табельный номер', 'Штатная должность']] # столбцы которые нам надо
    people = people.sort_values('full_name')
    n = people.shape[0] # n - кол-во людей 
    r = first_row       # r - текущая строчка, линия в excel
    
    for person in people.itertuples():
        sheet.cell(r, 1).value = person[1]  # full_name
        sheet.cell(r, 5).value = person[2]  # Табельный номер
        sheet.cell(r, 6).value = person[3]  # Штатная должность
        r += 2 #добавляем через две клетки нового сотрудника 
     
    # удаляем неиспользованные строки 
    sheet.delete_rows(r, last_row + 1  - r)

    #корректируем print area чтобы она соотвестовала финальному результату, который нам нужен (без пустых пропусков и т.п)
    sheet.print_area = sheet.calculate_dimension()

    wb.save(file)
    # нужен для теста (сделает один файл), удалите break если надо чтобы сделало все департаменты (весь цикл)
    #break